In [41]:
import os
import json
import pandas as pd
import numpy as np
import gc
import joblib
import psutil
import torch
from pathlib import Path
import torch.optim as optim
import matplotlib.pyplot as plt
from rdkit import Chem, DataStructs
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import (
     accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, 
     average_precision_score, mean_absolute_error, mean_squared_error, r2_score, 
     confusion_matrix, ConfusionMatrixDisplay)
from sklearn.model_selection import train_test_split
from lightning import pytorch as pl
from chemprop import data, featurizers, models, nn
from chemprop.models.utils import save_model
from chemprop.data import MoleculeDatapoint, MoleculeDataset, build_dataloader
from chemprop.featurizers import SimpleMoleculeMolGraphFeaturizer
from chemprop.models import MPNN
from chemprop.nn import BondMessagePassing, MeanAggregation, BinaryClassificationFFN, RegressionFFN

# Directory Setup

In [42]:
base_dir = '/Users/Avi/Dissertation'

data_dirs = {
    "train": os.path.join(base_dir, 'Data/Curated/Train_Data'),
    "scaffold_test": os.path.join(base_dir, 'Data/Curated/Test_Data/Scaffold_Split'),
    "random_test": os.path.join(base_dir, 'Data/Curated/Test_Data/Random_Split'),
}

models_dirs = {
    "rf_classification_models": os.path.join(base_dir, 'Models/Curated/Random_Forest/Classification'),
    "rf_regression_models": os.path.join(base_dir, 'Models/Curated/Random_Forest/Regression'),
    "chemprop_classification_models": os.path.join(base_dir, 'Models/Curated/ChemProp/Classification'),
    "chemprop_regression_models": os.path.join(base_dir, 'Models/Curated/ChemProp/Regression'),
}

results_dirs = {
    "rf_classification_results": os.path.join(base_dir, 'Results/Model_Performance/Curated/Random_Forest/Classification'),
    "rf_regression_results": os.path.join(base_dir, 'Results/Model_Performance/Curated/Random_Forest/Regression'),
    "chemprop_classification_results": os.path.join(base_dir, 'Results/Model_Performance/Curated/ChemProp/Classification'),
    "chemprop_regression_results": os.path.join(base_dir, 'Results/Model_Performance/Curated/ChemProp/Regression'),
}

prediction_dirs = {
    "rf_classification_matrices": os.path.join(base_dir, 'Results/Model_Performance/Curated/Random_Forest/Classification/Confusion_Matrices'),
    "rf_regression_plots": os.path.join(base_dir, 'Results/Model_Performance/Curated/Random_Forest/Regression/Pred_VS_Actual_Plots'),
    "chemprop_classification_matrices": os.path.join(base_dir, 'Results/Model_Performance/Curated/ChemProp/Classification/Confusion_Matrices'),
    "chemprop_regression_plots": os.path.join(base_dir, 'Results/Model_Performance/Curated/ChemProp/Regression/Pred_VS_Actual_Plots'),
}

hyperparams_dir = {
    "curated_rf_classification_hyperparameters": os.path.join(base_dir, 'Models/Random_Forest_Parameters/Curated/Classification'),
    "curated_rf_regression_hyperparameters": os.path.join(base_dir, 'Models/Random_Forest_Parameters/Curated/Regression')
}

for directory in {**models_dirs, **results_dirs, **prediction_dirs}.values():
    os.makedirs(directory, exist_ok=True)

# Assigning Targets and Fingerprints

In [51]:
targets = ['CHEMBL4078', 'CHEMBL279', 'CHEMBL5763', 'CHEMBL240', 'CHEMBL4005']

fingerprints = ['ECFP', 'MACCS Keys', 'AP2', 'AP3', 'AP2+AP3']

# Defining Helper Functions

In [44]:
def load_fingerprinted_data(file_path):
    df = pd.read_csv(file_path)
    for fp in fingerprints:
        df[fp] = df[fp].apply(DataStructs.CreateFromBitString)
    return df

def convert_fp(fp):
    arr = np.zeros((1,), dtype=np.int32)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

def prepare_rf_data(df, fingerprint, target_column):
    X = np.array([convert_fp(fp) for fp in df[fingerprint]])
    y = df[target_column].values
    return X, y

def prepare_chemprop_data(df, target_column):
    df = df[['canonical_smiles', target_column]].copy()
    df.columns = ['smiles', 'target']
    return df

def load_hyperparameters(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def save_confusion_matrix(y_true, y_pred, output_path):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.savefig(output_path)
    plt.close()

def save_pred_vs_actual_plot(y_true, y_pred, output_path):
    plt.figure(figsize=(8, 8))
    plt.scatter(y_true, y_pred, alpha=0.5)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], color='red', linestyle='--')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title('Predicted vs Actual')
    plt.savefig(output_path)
    plt.close()

def cleanup_torch_resources():
    for proc in psutil.process_iter(['pid', 'name']):
        if proc.info['name'] == 'torch_shm_manager':
            psutil.Process(proc.info['pid']).terminate()
    torch.cuda.empty_cache()
    gc.collect()

# Defining Functions to Train and Evaluate Random Forest Models

In [45]:
def train_rf_model(X_train, y_train, model_path, hyperparams, classification=True):
    if classification:
        model = RandomForestClassifier(**hyperparams, random_state=42, n_jobs=-1)
    else:
        model = RandomForestRegressor(**hyperparams, random_state=42, n_jobs=-1)
    
    model.fit(X_train, y_train)
    joblib.dump(model, model_path)
    return model

def evaluate_rf_model(model, X, y, classification=True, plot_dir=None, target_name=""):
    y_pred = model.predict(X)
    
    if classification:
        y_prob = model.predict_proba(X)[:, 1]
        metrics = {
            "accuracy": accuracy_score(y, y_pred),
            "precision": precision_score(y, y_pred),
            "recall": recall_score(y, y_pred),
            "f1": f1_score(y, y_pred),
            "auroc": roc_auc_score(y, y_prob),
            "auprc": average_precision_score(y, y_prob)
        }

        if plot_dir:
            output_path = os.path.join(plot_dir, f'{target_name}_Confusion_Matrix.png')
            save_confusion_matrix(y, y_pred, output_path, title=f'{target_name} Confusion Matrix')
    else:
        metrics = {
            "mae": mean_absolute_error(y, y_pred),
            "mse": mean_squared_error(y, y_pred),
            "rmse": np.sqrt(mean_squared_error(y, y_pred)),
            "r2": r2_score(y, y_pred)
        }
        
        if plot_dir:
            output_path = os.path.join(plot_dir, f'{target_name}_Predicted_VS_Actual.png')
            save_regression_plot(y, y_pred, output_path, title=f'{target_name} Predicted VS Actual')
    
    return metrics

# Defining Functions to Train and Evaluate ChemProp Models

In [46]:
def train_chemprop_model(train_df, val_df, model_path, hyperparams):
    train_data = [MoleculeDatapoint.from_smi(smi, [y]) for smi, y in zip(train_df['smiles'], train_df['target'])]
    val_data = [MoleculeDatapoint.from_smi(smi, [y]) for smi, y in zip(val_df['smiles'], val_df['target'])]
    
    featurizer = SimpleMoleculeMolGraphFeaturizer()
    train_dset = MoleculeDataset(train_data, featurizer)
    val_dset = MoleculeDataset(val_data, featurizer)

    class_balance = not hyperparams['regression']

    train_loader = build_dataloader(train_dset, batch_size=hyperparams['batch_size'], class_balance=class_balance, num_workers=4, shuffle=True, persistent_workers=True)
    val_loader = build_dataloader(val_dset, batch_size=hyperparams['batch_size'], num_workers=4, shuffle=False, persistent_workers=True)

    mp = BondMessagePassing()
    agg = MeanAggregation()
    
    ffn = RegressionFFN(dropout=hyperparams['dropout']) if hyperparams['regression'] else BinaryClassificationFFN(dropout=hyperparams['dropout'])
    
    batch_norm = True
    mpnn = MPNN(mp, agg, ffn, batch_norm)
    
    optimizer = optim.AdamW(mpnn.parameters(), lr=hyperparams['learning_rate'], weight_decay=hyperparams['weight_decay'])
    mpnn.configure_optimizers = lambda: optimizer

    trainer = pl.Trainer(
        logger=False,
        enable_checkpointing=False,
        enable_progress_bar=True,
        accelerator="cpu",
        devices=1,
        max_epochs=hyperparams['max_epochs']
    )

    trainer.fit(mpnn, train_loader, val_loader)
    save_model(model_path, mpnn)
    return mpnn

def evaluate_chemprop_model(model, test_df, regression=False, plot_dir=None, target_name=""):
    test_data = [MoleculeDatapoint.from_smi(smi, [y]) for smi, y in zip(test_df['smiles'], test_df['target'])]
    featurizer = SimpleMoleculeMolGraphFeaturizer()
    test_dset = MoleculeDataset(test_data, featurizer)
    test_loader = build_dataloader(test_dset, num_workers=4, shuffle=False, persistent_workers=True)

    trainer = pl.Trainer(logger=False, enable_checkpointing=False)
    
    with torch.inference_mode():
        test_preds = trainer.predict(model, test_loader)
        test_preds = np.concatenate(test_preds, axis=0)
    
    if regression:
        predictions = test_preds
        y_true = test_df['target'].values.astype(np.float64)

        metrics = {
            "mae": mean_absolute_error(y_true, predictions),
            "mse": mean_squared_error(y_true, predictions),
            "rmse": np.sqrt(mean_squared_error(y_true, predictions)),
            "r2": r2_score(y_true, predictions)
        }
        
        if plot_dir:
            output_path = os.path.join(plot_dir, f'{target_name}_Predicted_VS_Actual.png')
            save_regression_plot(y_true, predictions, output_path, title=f'{target_name} Predicted VS Actual')

        return predictions, metrics

    else:
        predictions = (test_preds > 0.5).astype(int)
        y_true = test_df['target'].values
        
        metrics = {
            "accuracy": accuracy_score(y_true, predictions),
            "precision": precision_score(y_true, predictions),
            "recall": recall_score(y_true, predictions),
            "f1": f1_score(y_true, predictions),
            "auroc": roc_auc_score(y_true, test_preds),
            "auprc": average_precision_score(y_true, test_preds)
        }
        
        if plot_dir:
            output_path = os.path.join(plot_dir, f'{target_name}_Confusion_Matrix.png')
            save_confusion_matrix(y_true, predictions, output_path, title=f'{target_name} Confusion Matrix')

        return predictions, metrics

# Executing Functions

## Random Forest Classification

In [16]:
for target in targets:
    train_file_path = os.path.join(data_dirs["train"], f'{target}_Train.csv')
    scaffold_test_file_path = os.path.join(data_dirs["scaffold_test"], f'{target}_Test_Scaffold.csv')
    random_test_file_path = os.path.join(data_dirs["random_test"], f'{target}_Test_Random.csv')

    train_df = load_fingerprinted_data(train_file_path)
    scaffold_test_df = load_fingerprinted_data(scaffold_test_file_path)
    random_test_df = load_fingerprinted_data(random_test_file_path)

    target_matrix_dir = os.path.join(prediction_dirs['rf_classification_matrices'], target)
    os.makedirs(target_matrix_dir, exist_ok=True)

    all_results = []

    for fingerprint in fingerprints:
        X_train, y_train = prepare_rf_data(train_df, fingerprint, 'activity')
        X_scaffold_test, y_scaffold_test = prepare_rf_data(scaffold_test_df, fingerprint, 'activity')
        X_random_test, y_random_test = prepare_rf_data(random_test_df, fingerprint, 'activity')

        hyperparams_file_classification = os.path.join(hyperparams_dir['curated_rf_classification_hyperparameters'], f'{target}_{fingerprint}_Best_RF_Curated_Classification_Hyperparams.json')
        hyperparams_classification = load_hyperparameters(hyperparams_file_classification)

        model_path = os.path.join(models_dirs["rf_classification_models"], f'{target}_{fingerprint}_RF_Curated_Classification_Model.pkl')
        
        model = train_rf_model(X_train, y_train, model_path, hyperparams_classification, classification=True)
        
        train_metrics = evaluate_rf_model(model, X_train, y_train, classification=True)
        scaffold_test_metrics = evaluate_rf_model(model, X_scaffold_test, y_scaffold_test, classification=True)
        random_test_metrics = evaluate_rf_model(model, X_random_test, y_random_test, classification=True)

        save_confusion_matrix(y_train, model.predict(X_train), 
                              output_path=os.path.join(target_matrix_dir, f'{fingerprint}_Train_Confusion_Matrix.png'))

        save_confusion_matrix(y_scaffold_test, model.predict(X_scaffold_test), 
                              output_path=os.path.join(target_matrix_dir, f'{fingerprint}_Scaffold_Confusion_Matrix.png'))

        save_confusion_matrix(y_random_test, model.predict(X_random_test), 
                              output_path=os.path.join(target_matrix_dir, f'{fingerprint}_Random_Confusion_Matrix.png'))

        all_results.extend([
            {"target": target, "fingerprint": fingerprint, "split": "train", **train_metrics},
            {"target": target, "fingerprint": fingerprint, "split": "scaffold", **scaffold_test_metrics},
            {"target": target, "fingerprint": fingerprint, "split": "random", **random_test_metrics}
        ])

    results_df = pd.DataFrame(all_results)
    results_df.to_csv(os.path.join(results_dirs["rf_classification_results"], f'{target}_RF_Curated_Classification_Results.csv'), index=False)

## Random Forest Regression

In [17]:
for target in targets:
    train_file_path = os.path.join(data_dirs["train"], f'{target}_Train.csv')
    scaffold_test_file_path = os.path.join(data_dirs["scaffold_test"], f'{target}_Test_Scaffold.csv')
    random_test_file_path = os.path.join(data_dirs["random_test"], f'{target}_Test_Random.csv')

    train_df = load_fingerprinted_data(train_file_path)
    scaffold_test_df = load_fingerprinted_data(scaffold_test_file_path)
    random_test_df = load_fingerprinted_data(random_test_file_path)

    target_plot_dir = os.path.join(prediction_dirs['rf_regression_plots'], target)
    os.makedirs(target_plot_dir, exist_ok=True)

    all_results = []

    for fingerprint in fingerprints:
        X_train, y_train = prepare_rf_data(train_df, fingerprint, 'pchembl_value')
        X_scaffold_test, y_scaffold_test = prepare_rf_data(scaffold_test_df, fingerprint, 'pchembl_value')
        X_random_test, y_random_test = prepare_rf_data(random_test_df, fingerprint, 'pchembl_value')

        hyperparams_file_regression = os.path.join(hyperparams_dir['curated_rf_regression_hyperparameters'], f'{target}_{fingerprint}_Best_RF_Curated_Regression_Hyperparams.json')
        hyperparams_regression = load_hyperparameters(hyperparams_file_regression)

        model_path = os.path.join(models_dirs["rf_regression_models"], f'{target}_{fingerprint}_RF_Curated_Regression_Model.pkl')
        
        model = train_rf_model(X_train, y_train, model_path, hyperparams_regression, classification=False)
        
        train_metrics = evaluate_rf_model(model, X_train, y_train, classification=False)
        scaffold_test_metrics = evaluate_rf_model(model, X_scaffold_test, y_scaffold_test, classification=False)
        random_test_metrics = evaluate_rf_model(model, X_random_test, y_random_test, classification=False)

        save_pred_vs_actual_plot(y_train, model.predict(X_train), 
                                 output_path=os.path.join(target_plot_dir, f'{fingerprint}_Train_Pred_VS_Actual.png'))

        save_pred_vs_actual_plot(y_scaffold_test, model.predict(X_scaffold_test), 
                                 output_path=os.path.join(target_plot_dir, f'{fingerprint}_Scaffold_Pred_VS_Actual.png'))

        save_pred_vs_actual_plot(y_random_test, model.predict(X_random_test), 
                                 output_path=os.path.join(target_plot_dir, f'{fingerprint}_Random_Pred_VS_Actual.png'))

        all_results.extend([
            {"target": target, "fingerprint": fingerprint, "split": "train", **train_metrics},
            {"target": target, "fingerprint": fingerprint, "split": "scaffold", **scaffold_test_metrics},
            {"target": target, "fingerprint": fingerprint, "split": "random",  **random_test_metrics}
        ])

    results_df = pd.DataFrame(all_results)
    results_df.to_csv(os.path.join(results_dirs["rf_regression_results"], f'{target}_RF_Curated_Regression_Results.csv'), index=False)

## ChemProp Classification

In [50]:
for target in targets:
    train_file_path = os.path.join(data_dirs["train"], f'{target}_Train.csv')
    scaffold_test_file_path = os.path.join(data_dirs["scaffold_test"], f'{target}_Test_Scaffold.csv')
    random_test_file_path = os.path.join(data_dirs["random_test"], f'{target}_Test_Random.csv')

    train_df = pd.read_csv(train_file_path)
    scaffold_test_df = pd.read_csv(scaffold_test_file_path)
    random_test_df = pd.read_csv(random_test_file_path)

    train_df = prepare_chemprop_data(train_df, 'activity')
    scaffold_test_df = prepare_chemprop_data(scaffold_test_df, 'activity')
    random_test_df = prepare_chemprop_data(random_test_df, 'activity')

    train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

    hyperparams = {
        'regression': False,
        'max_epochs': 100,
        'learning_rate': 1e-3,
        'dropout': 0.3,
        'batch_size': 128,
        'weight_decay': 1e-3
    }

    model_path = os.path.join(models_dirs["chemprop_classification_models"], f'{target}_ChemProp_Curated_Classification_Model.pt')
    model = train_chemprop_model(train_df, val_df, model_path, hyperparams)

    train_predictions, train_metrics = evaluate_chemprop_model(model, train_df, regression=False)
    scaffold_test_predictions, scaffold_test_metrics = evaluate_chemprop_model(model, scaffold_test_df, regression=False)
    random_test_predictions, random_test_metrics = evaluate_chemprop_model(model, random_test_df, regression=False)

    target_matrix_dir = os.path.join(prediction_dirs['chemprop_classification_matrices'], target)
    os.makedirs(target_matrix_dir, exist_ok=True)

    save_confusion_matrix(train_df['target'].values, train_predictions, 
                          os.path.join(target_matrix_dir, f'Train_Confusion_Matrix.png'))

    save_confusion_matrix(scaffold_test_df['target'].values, scaffold_test_predictions, 
                          os.path.join(target_matrix_dir, f'Scaffold_Confusion_Matrix.png'))

    save_confusion_matrix(random_test_df['target'].values, random_test_predictions, 
                          os.path.join(target_matrix_dir, f'Random_Confusion_Matrix.png'))

    results = [
        {"target": target, "split": "train", **train_metrics},
        {"target": target, "split": "scaffold", **scaffold_test_metrics},
        {"target": target, "split": "random", **random_test_metrics}
    ]

    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(results_dirs["chemprop_classification_results"], f'{target}_ChemProp_Curated_Classification_Results.csv'), index=False)

    cleanup_torch_resources()

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/MSC/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name            | Type                    | Params | Mode 
--------------------------------------------------------------------
0 | message_passing | BondMessagePassing      | 227 K  | train
1 | agg             | MeanAggregation         | 0      | train
2 | bn              | BatchNorm1d             | 600    | train
3 | predictor       | BinaryClassificationFFN | 90.6 K | train
4 | X_d_transform   | Identity                | 0      | train
--------------------------------------------------------------------
318 K     Trainable params
0         Non-trainable params
318 K     Total params
1.276     Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Predicting: |                                             | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |                                             | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |                                             | 0/? [00:00<?, ?it/s]

## ChemProp Regression

In [48]:
for target in targets:
    train_file_path = os.path.join(data_dirs["train"], f'{target}_Train.csv')
    scaffold_test_file_path = os.path.join(data_dirs["scaffold_test"], f'{target}_Test_Scaffold.csv')
    random_test_file_path = os.path.join(data_dirs["random_test"], f'{target}_Test_Random.csv')

    train_df = pd.read_csv(train_file_path)
    scaffold_test_df = pd.read_csv(scaffold_test_file_path)
    random_test_df = pd.read_csv(random_test_file_path)

    train_df = prepare_chemprop_data(train_df, 'pchembl_value')
    scaffold_test_df = prepare_chemprop_data(scaffold_test_df, 'pchembl_value')
    random_test_df = prepare_chemprop_data(random_test_df, 'pchembl_value')

    train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

    hyperparams = {
        'regression': True,
        'max_epochs': 100,
        'learning_rate': 1e-3,
        'dropout': 0.3,
        'batch_size': 128,
        'weight_decay': 1e-3
    }

    model_path = os.path.join(models_dirs["chemprop_regression_models"], f'{target}_ChemProp_Curated_Regression_Model.pt')
    model = train_chemprop_model(train_df, val_df, model_path, hyperparams)

    train_predictions, train_metrics = evaluate_chemprop_model(model, train_df, regression=True)
    scaffold_test_predictions, scaffold_test_metrics = evaluate_chemprop_model(model, scaffold_test_df, regression=True)
    random_test_predictions, random_test_metrics = evaluate_chemprop_model(model, random_test_df, regression=True)

    target_plots_dir = os.path.join(prediction_dirs['chemprop_regression_plots'], target)
    os.makedirs(target_plots_dir, exist_ok=True)

    save_pred_vs_actual_plot(train_df['target'].values, train_predictions, 
                             os.path.join(target_plots_dir, f'Train_Pred_VS_Actual.png'))
    
    save_pred_vs_actual_plot(scaffold_test_df['target'].values, scaffold_test_predictions, 
                             os.path.join(target_plots_dir, f'Scaffold_Pred_VS_Actual.png'))
    
    save_pred_vs_actual_plot(random_test_df['target'].values, random_test_predictions, 
                             os.path.join(target_plots_dir, f'Random_Pred_VS_Actual.png'))

    results = [
        {"target": target, "split": "train", **train_metrics},
        {"target": target, "split": "scaffold", **scaffold_test_metrics},
        {"target": target, "split": "random", **random_test_metrics}
    ]

    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(results_dirs["chemprop_regression_results"], f'{target}_ChemProp_Curated_Regression_Results.csv'), index=False)

    cleanup_torch_resources()

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/MSC/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | train
3 | predictor       | RegressionFFN      | 90.6 K | train
4 | X_d_transform   | Identity           | 0      | train
---------------------------------------------------------------
318 K     Trainable params
0         Non-trainable params
318 K     Total params
1.276     Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Predicting: |                                             | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |                                             | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |                                             | 0/? [00:00<?, ?it/s]